In [1]:
import xml.etree.ElementTree as ET
from pathlib import Path
import shutil

class ImageInteractor:
    def __init__(self, dataset_path: str, target: str, category: str = None):
        self.dataset_path = Path(dataset_path)
        self.target = target
        self.category = category
        self.imgs = self.get_images()

    def get_images(self):
        target_path = self.dataset_path / self.target
        imgs = []
        if not self.category:
            imgs = [image for image in target_path.glob("**/*") if image.suffix in [".jpg"]]
        else:
            imgs = self.get_images_by_category(target_path)
        return sorted(imgs)

    def get_images_by_category(self, target_path):
        imgs = []
        for file in target_path.glob("**/*"):
            if file.suffix in [".xml"]:
                tree = ET.parse(file)
                root = tree.getroot()
                for obj in root.findall("object"):
                    if obj.find('property/category').text == self.category:
                        img = file.with_suffix(".jpg")
                        if img.exists():
                            imgs.append(img)
        return imgs

    def gather_images_by_category(self, destination_folder: str, min_width: int, min_height: int):
        # Create the destination folder if it doesn't exist
        destination_path = Path(destination_folder)
        destination_path.mkdir(parents=True, exist_ok=True)

        # Iterate through the images and copy them to the destination folder if they meet the category condition and bounding box size
        for img in self.imgs:
            xml = img.with_suffix(".xml")
            if xml.exists():
                tree = ET.parse(xml)
                root = tree.getroot()

                # Flag to indicate if any valid bounding box is found
                valid_bbox_found = False

                for obj in root.findall("object"):
                    if obj.find('property/category').text == self.category:
                        ymax = int(obj.find('bndbox/ymax').text)
                        xmin = int(obj.find('bndbox/xmin').text)
                        ymin = int(obj.find('bndbox/ymin').text)
                        xmax = int(obj.find('bndbox/xmax').text)

                        bbox_width = xmax - xmin + 1
                        bbox_height = ymax - ymin + 1

                        if bbox_width < min_width or bbox_height < min_height:
                            valid_bbox_found = False
                            break
                        else:
                            valid_bbox_found = True

                if valid_bbox_found:
                    new_path = destination_path / img.name
                    shutil.copy(str(img), str(new_path))
                    print(f"Copied {img.name} to {destination_folder}.")
                else:
                    print(f"No valid bounding box found for {img.name}. Skipping copy.")

        print(f"Images with category '{self.category}' and bbox width >= {min_width}, and bbox height >= {min_height} copied to '{destination_folder}'.")



In [ ]:
#set dataset_path, target_folder(name), target_category, and destination_folder
dataset_path = "/mnt/disks/data1/aihub/Training/"

target_folder = "남해_여수항_1구역_BOX"

target_category = "등대"

destination_folder = "/mnt/disks/data1/aihub/Training/이미지/둥대/남해_여수항_1구역"

interactor = ImageInteractor(dataset_path, target_folder, target_category)

# Set minimum size with mid_width&mid_height
interactor.gather_images_by_category(destination_folder, min_width=25, min_height=40)

등대(25*40),부표(30*30),기타부유물(50*15)